In [1]:
import numpy as np
from numpy.linalg import matrix_power as mpow

In [2]:
#Grafo de prueba, A-B y A-C
A = np.array([[0,1,1],[1,0,0],[0,0,1]])
A

array([[0, 1, 1],
       [1, 0, 0],
       [0, 0, 1]])

In [3]:
#Primero enumeracion simple (grafos sin direccion)
#Las restricciones son un diccionario con las clases como llave,
#con sets indicando que elementos no estan permitidos
def enumeracion(A, restricciones, MAX, symbs):
    n = len(A)
    
    #Inicializamos una matriz para el proceso en zeros
    S = np.zeros((n,n))
    S[0] = [sum(a) for a in A]
    
    
    #Arreglo para checar los nodos restantes
    left = [i for i in range(n)]
    
    #Se inicializa la solucion como un diccionario vacio
    soln = {}
        
    
    if MAX > 0:
        #Modo con maximo
        #Se rellena la solucion y la restriccion con las llavesd que les faltan
        existentes = restricciones.keys()
        
        try:
            nxt = max(existentes) + 1
        except Exception:
            nxt = 0
        
        for i in range(nxt):
            soln[i] = []
            if i not in existentes:
                existentes[i] = set()
        
        for i in range(n):
            #Inicializamos maximo en -infinito e indice en -1
            m = -np.inf
            index = -1

            #Buscamos el maximo en los nodos restantes
            for j in left:
                if S[i,j] > m :
                    m = S[i,j]
                    index = j

            #Nuesta solucion es index, la quitamos de los nodos disponibles
            left.remove(index)

            #El siguiente renglon es el mismo que el pasado menos las conexiones de index
            if i-1<m:
                S[i+1] = S[i] - A[index]
            
            posibles = [key for key, value in restricciones.items() if ( len(soln[key]) < MAX and \
                                                                       index not in value )]
            if len(posibles) > 0 :
                #Encuentra el posible con menos elementos
                clase = posibles[0]
                m = len(soln[clase])
                
                for p in posibles:
                    P = len(soln[p])
                    if  P < m :
                        clase = p
                        m = P
                
                restricciones[clase] = restricciones[clase] | set(np.where(A[index] == 1)[0])
                soln[clase].append(symbs[index])
                
            else:
                #No hay posible, crea otra clase
                restricciones[nxt] = set(np.where(A[index] == 1)[0])
                soln[nxt] = [symbs[index]]
                nxt = nxt + 1
        
    else:
        #Modo sin maximo
        
        for i in range(n):
            #Inicializamos maximo en -infinito e indice en -1
            m = -np.inf
            index = -1

            #Buscamos el maximo en los nodos restantes
            for j in left:
                if S[i,j] > m :
                    m = S[i,j]
                    index = j

            #Nuesta solucion es index, la quitamos de los nodos disponibles
            left.remove(index)

            #El siguiente renglon es el mismo que el pasado menos las conexiones de index
            if i-1<m:
                S[i+1] = S[i] - A[index]

            #Agregamos index a su clase, tomando en cuenta las restricciones puestas, intentando primero en la clase 0
            clase = 0
            flag = False

            #Busca la clase
            while not flag:
                try:
                    if index not in restricciones[clase]:
                        flag = True
                        restricciones[clase] = restricciones[clase] | set(np.where(A[index] == 1)[0])

                    else:
                        clase += 1

                except Exception:
                    flag = True
                    restricciones[clase] = set(np.where(A[index] == 1)[0])

            #Guarda j en la clase
            try:
                soln[clase].append(symbs[index])
            except Exception:
                soln[clase] = [symbs[index]]
    
    return soln

In [4]:
print('Enumeracion Simple')
syms = input('Escriba todos los nodos, separados por comas: \n')

symbs = syms.replace(' ','').split(',')
n = len(symbs)
syms = { symbs[i] : i for i in range(n) }
A = np.zeros((n,n))

print('Escriba las conexiones, separando los elementos com comas:')

flag = True
while flag:
    s = input('Clan: \n')
    s = s.replace(' ' ,'').split(',')
    
    for i in s:
        for j in [k for k in s if k!=i]:
            A[syms[i],syms[j]] = 1
    flag = (input('Hay otra conexión? (S/N): \n') == 'S')

MAX = int(input ('Ingrese el número máximo de elementos por clase (si no hay ingrese un numero negativo)'))

restricciones = {}

while (input('Hay restricciones sobre los nodos? (S/N): \n') == 'S'):
    s = input('Restriccion con formato A >= n (clase de A mayor igual a n)\n')
    try:
        sym, index = s.replace(' ','').split('>=')
        index = int(index)
    except Exception:
        print('Restriccion invalida')
    
    for j in range(index):
        try:
            restricciones[j].add(sym)
        except Exception:
            restricciones[j] = set([sym])

Enumeracion Simple
Escriba todos los nodos, separados por comas: 
1,2,3,4,5,6
Escriba las conexiones, separando los elementos com comas:
Clan: 
1,2,3
Hay otra conexión? (S/N): 
n
Ingrese el número máximo de elementos por clase (si no hay ingrese un numero negativo)0
Hay restricciones sobre los nodos? (S/N): 
n


In [5]:
enumeracion(A,restricciones, MAX, symbs)

{0: ['1', '4', '5', '6'], 1: ['2'], 2: ['3']}